In [1]:
import numpy as np
import torch
import json

from libraries.model   import nGCNN, eGCNN, denoise, get_random_graph
from libraries.dataset import revert_standardize_dataset
from libraries.graph   import POSCAR_graph_encoding

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# From random noise, we generate completely new materials
# A target property can be seeked with this approach

In [3]:
# Define folder in which all data will be stored
target_folder    = 'models/GM_BiSI'
edge_model_name = f'{target_folder}/edge_model.pt'
node_model_name = f'{target_folder}/node_model.pt'

# Number of graphs to predict
N_predictions = 10

# Define target to be generated
target_tensor = None

# Load model data

In [4]:
# Read the file in JSON format to a dictionary
with open(f'{target_folder}/model_parameters.json', 'r') as json_file:
    model_parameters = json.load(json_file)

# Number of diffusing and denoising steps
n_t_steps = model_parameters['n_t_steps']

# Decay of parameter alpha
noise_contribution = model_parameters['noise_contribution']
alpha_decay = 0.5 * (1 - noise_contribution**2)

# Dropouts for node and edge models (independent of each other)
dropout_node = model_parameters['dropout_node']
dropout_edge = model_parameters['dropout_edge']

# Generation of graph database for training

Load the datasets, already standarized if possible.

In [5]:
labels_name                 = f'{target_folder}/labels.pt'
dataset_name                = f'{target_folder}/dataset.pt'
dataset_name_std            = f'{target_folder}/standardized_dataset.pt'
dataset_parameters_name_std = f'{target_folder}/standardized_dataset_parameters.json'  # Parameters for rescaling the predictions

# Load the standardized dataset, with corresponding labels and parameters
dataset = torch.load(dataset_name_std)
labels  = torch.load(labels_name)

# Read the file in JSON format to a dictionary
with open(dataset_parameters_name_std, 'r') as json_file:
    dataset_parameters = json.load(json_file)

In [6]:
# Calculate the mean and standard deviation of the number of nodes
total_nodes = torch.tensor([data.num_nodes for data in dataset])
mean_nodes  = torch.mean(total_nodes.float()).item()
std_nodes   = torch.std(total_nodes.float()).item()

mean_nodes, std_nodes

(72.0, 0.0)

# Loading the model

In [7]:
# Determine number of features in dataset, considering the t_step information
n_features = dataset[0].num_node_features + 1

# Instantiate the models for nodes and edges
node_model = nGCNN(n_features, dropout_node).to(device)
node_model.load_state_dict(torch.load(node_model_name))
node_model.eval()

edge_model = eGCNN(n_features, dropout_edge).to(device)
edge_model.load_state_dict(torch.load(edge_model_name))
edge_model.eval()

print('\nNode GCNN:')
print(node_model)
print('\nEdge GCNN:')
print(edge_model)


Node GCNN:
nGCNN(
  (conv1): GraphConv(5, 256)
  (conv2): GraphConv(256, 5)
)

Edge GCNN:
eGCNN(
  (linear1): Linear(in_features=6, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=1, bias=True)
)


# Generating new cystals

In [8]:
# Predicting loop
predicted_dataset = []
with torch.no_grad():
    for i in range(N_predictions):
        # Get random number of nodes
        n_nodes = int(np.random.normal(mean_nodes, std_nodes))
        
        # Get random graph, acting as diffused
        diffused_graph = get_random_graph(n_nodes, n_features-1)
        
        # Denoise the diffused graph
        #print(f'Denoising...')
        denoised_graph, _ = denoise(diffused_graph, n_t_steps, node_model, edge_model,
                                    s=alpha_decay, sigma=model_parameters['sigma'], target=target_tensor)
        
        # Append generated graph
        predicted_dataset.append(denoised_graph)

# Revert stardadization
denoised_graphs = revert_standardize_dataset(predicted_dataset, dataset_parameters)
denoised_graphs

[Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556])]